# cmdStan input simulator

In [1]:
from __future__ import (absolute_import, division, print_function, 
   unicode_literals, generators, nested_scopes, with_statement)
from builtins import (bytes, dict, int, list, object, range, str, ascii,
   chr, hex, input, next, oct, open, pow, round, super, filter, map, zip)
# The above imports should allow this program to run in both Python 2 and
# Python 3.  You might need to update your version of module "future".
import sys
import os
import math
import seaborn as sns
import matplotlib.pyplot as plt
import math
from sklearn.metrics import roc_auc_score
from sklearn import metrics
import numpy as np
import statistics

# Need to use package inside `Required_Python_Packages`
import ProgramName
from Rex import Rex
rex=Rex()
import TempFilename
from SummaryStats import SummaryStats
import getopt


In [2]:
def getFieldIndex(label,fields):
    numFields=len(fields)
    index=None
    for i in range(7,numFields):
        if(fields[i]==label): index=i
    return index

def getThetaIndex(fields):
    numFields=len(fields)
    thetaIndex=-1
    pIndex=-1
    qIndex=-1
    for i in range(7,numFields):
        if(fields[i]=="theta"): thetaIndex=i
        elif(fields[i]=="p"): pIndex=i
        elif(fields[i]=="q"): qIndex=i
    return (thetaIndex,pIndex,qIndex)

def writeToFile(fields,OUT):
    numFields=len(fields)
    for i in range(7,numFields):
        print(fields[i],end="",file=OUT)
        if(i<numFields-1): print("\t",end="",file=OUT)
    print(file=OUT)

def computeThetaDirectly(fields,pIndex,qIndex):
    #raise Exception("computeThetaDirectly")
    if(pIndex<0 or qIndex<0): return 1 # beta-binomial model
    p=float(fields[pIndex])
    q=float(fields[qIndex])
    theta=q/p/((1-q)/(1-p))
    return theta

def printFields(fields,hFile):
    numFields=len(fields)
    for i in range(7,numFields):
        print(i-6,"=",fields[i],sep="",end="",file=hFile)
        if(i<numFields-1): print("\t",end="",file=hFile)
    print(file=hFile)
import numpy as np
from sklearn import metrics

In [3]:
class Variant:
    def __init__(self,fields):
        self.v=fields[0]
        self.N_DNA=int(fields[4])
        rnaIndex=5+2*self.N_DNA
        self.N_RNA=int(fields[rnaIndex])
        self.a=Variant.getReadCounts(fields,6,self.N_DNA)
        self.b=Variant.getReadCounts(fields,5,self.N_DNA)
        self.k=Variant.getReadCounts(fields,rnaIndex+2,self.N_RNA)
        self.m=Variant.getReadCounts(fields,rnaIndex+1,self.N_RNA)
    @classmethod
    def getReadCounts(cls,fields,start,numReps):
        counts=[]
        for rep in range(numReps):
            counts.append(fields[start+rep*2])
        return counts

def writeInputsFile(variants,filename):
    N_VARIANTS=len(variants)
    if(N_VARIANTS<1): raise Exception("no variants")
    OUT=open(filename,"wt")
    print("N_VARIANTS <- ",N_VARIANTS,sep="",file=OUT)
    print("v <- structure(c(",end="",file=OUT)
    for i in range(N_VARIANTS):
        print(round(float(variants[i].v),5),end="",file=OUT)
        if(i+1<N_VARIANTS): print(", ",end="",file=OUT)
    print("), .Dim=c(",N_VARIANTS,"))",sep="",file=OUT)
    N_DNA=variants[0].N_DNA
    print("N_DNA <- ",N_DNA,sep="",file=OUT)
    print("a <- structure(c(",end="",file=OUT)
    for j in range(N_DNA):
        for i in range(N_VARIANTS):
            print(variants[i].a[j],end="",file=OUT)
            if(i+1<N_VARIANTS): print(",",end="",file=OUT)
        if(j+1<N_DNA): print(",",end="",file=OUT)
    print("), .Dim=c(",N_VARIANTS,",",N_DNA,"))",sep="",file=OUT)
    print("b <- structure(c(",end="",file=OUT)
    for j in range(N_DNA):
        for i in range(N_VARIANTS):
            print(variants[i].b[j],end="",file=OUT)
            if(i+1<N_VARIANTS): print(",",end="",file=OUT)
        if(j+1<N_DNA): print(",",end="",file=OUT)
    print("), .Dim=c(",N_VARIANTS,",",N_DNA,"))",sep="",file=OUT)
    N_RNA=variants[0].N_RNA
    print("N_RNA <- ",N_RNA,sep="",file=OUT)
    print("k <- structure(c(",end="",file=OUT)
    for j in range(N_RNA):
        for i in range(N_VARIANTS):
            print(variants[i].k[j],end="",file=OUT)
            if(i+1<N_VARIANTS): print(",",end="",file=OUT)
        if(j+1<N_RNA): print(",",end="",file=OUT)
    print("), .Dim=c(",N_VARIANTS,",",N_RNA,"))",sep="",file=OUT)
    print("m <- structure(c(",end="",file=OUT)
    for j in range(N_RNA):
        for i in range(N_VARIANTS):
            print(variants[i].m[j],end="",file=OUT)
            if(i+1<N_VARIANTS): print(",",end="",file=OUT)
        if(j+1<N_RNA): print(",",end="",file=OUT)
    print("), .Dim=c(",N_VARIANTS,",",N_RNA,"))",sep="",file=OUT)
    OUT.close()
    


## - Simulate mixed regulatory and null variants 

In [86]:
#define variable 
# theta(effect size)
l=2
# Number of variants total
num = 1000
# Number of regulatory variants 
pos = 50
null= num-pos
per = pos/(pos+null)
# Minor Allele Frequency 
v= 0.1
# pre-defined 
qc = 124.6
# pre-defined
pc = 71.9
# Number of DNA site, usually 1
dna = 1
# Number of RNA site
rna = 10
# DNA reads 
dna_read = 30 #30 - 100 - 500 - 1000
# RNA reads (reads of all RNA pairs total)
rna_read = 300 #30 - 100 - 500 - 1000

file1 = "regulatory_v.txt".format(l,per,pos,dna_read)
file2 = "null_v.txt".format(1,per,null,dna_read)
file3 = "{}_data_{}_{}_read{}.txt".format(l,per,num,3030)


In [87]:
cmd1 = "./sim-equal.R {} {} {} {} {} {} {} {} {} > ./STANINPUTS/{}".format(pos,v,qc,pc,dna,rna,l,dna_read,rna_read,file1)
cmd2 = "./sim-equal.R {} {} {} {} {} {} {} {} {} > ./STANINPUTS/{}".format(null,v,qc,pc,dna,rna,1,dna_read,rna_read,file2)
cmd3 = "cat ./STANINPUTS/{} ./STANINPUTS/{} > ./STANINPUTS/{}".format(file1, file2, file3)
cmd4 = "rm ./STANINPUTS/{}".format(file1)
cmd5 = "rm ./STANINPUTS/{}".format(file2)


os.system(cmd1)
os.system(cmd2)
os.system(cmd3)
os.system(cmd4)
os.system(cmd5)


thetaIndex=None; pIndex=None; qIndex=None
variantIndex=0
variants=[]
with open("./STANINPUTS/{}".format(file3),"rt") as IN:
    for line in IN:
        # Check whether this variant is in the range to be processed
        if(variantIndex<0):
            variantIndex+=1
            continue
        elif(variantIndex>num): break
        fields=line.rstrip().split()
        variant=Variant(fields)
        variants.append(variant)
        variantIndex+=1

inputfile = './STANINPUTS/{}.staninputs'.format(file3)
writeInputsFile(variants, inputfile)

## - Simulate all regulatory variants

In [37]:
#define variable 
l=2
num = 1000
# for all regulatory variants simulation, per always = 1.0
per = 1.0
v= 0.1
qc = 124.6
pc = 71.9
dna = 1
rna = 10
# for the reads, when filename contains "5050", which means 
# for each DNA and RNA pair, reads are 50. 
dna_read = 100 #30 - 100 - 500 - 1000
rna_read = 1000 #30 - 100 - 500 - 1000
file3 = "{}_data_{}_{}_read{}{}.txt".format(l,per,num,dna_read,dna_read)

cmd1 = "./sim-equal.R {} {} {} {} {} {} {} {} {} > ./STANINPUTS/{}".format(num,v,qc,pc,dna,rna,l,dna_read,rna_read,file3)
os.system(cmd1)


0

In [38]:
thetaIndex=None; pIndex=None; qIndex=None
variantIndex=0
variants=[]
with open("./STANINPUTS/{}".format(file3),"rt") as IN:
    for line in IN:
        # Check whether this variant is in the range to be processed
        if(variantIndex<0):
            variantIndex+=1
            continue
        elif(variantIndex>num): break
        fields=line.rstrip().split()
        variant=Variant(fields)
        variants.append(variant)
        variantIndex+=1

inputfile = './STANINPUTS/{}.staninputs'.format(file3)
writeInputsFile(variants, inputfile)